# 数据分析：不同 label 带来的性能提升
本分析旨在比较 CSV 文件中 `label` 列的不同值（如 `RVV` 和 `RV`）在相同维度下的性能提升。

In [18]:
# 导入必要的库
import pandas as pd


In [19]:
# 读取 CSV 文件
file_path = "gem5-SimSecond_vs_vlen_dim3072_topk100.csv"
df = pd.read_csv(file_path)
print(f"加载文件: {file_path}")
print(df.head())


加载文件: gem5-SimSecond_vs_vlen_dim3072_topk100.csv
   vlen  SimSecond label
0   128   1.879889   RVV
1   256   1.418192   RVV
2   512   1.234390   RVV
3  1024   1.182808   RVV
4   128   2.295064    RV


In [20]:
# 定义一个函数来计算加速比 (基线时间 / RVV 时间)
def calculate_speedup(df, vlen_val, base_label, rvv_label):
    base_time = df[(df['vlen'] == vlen_val) & (df['label'] == base_label)]['SimSecond'].iloc[0]
    rvv_time = df[(df['vlen'] == vlen_val) & (df['label'] == rvv_label)]['SimSecond'].iloc[0]
    if pd.notna(base_time) and pd.notna(rvv_time) and rvv_time != 0:
        return base_time / rvv_time
    return None


In [21]:
# 计算不同配置和 VLEN 下的加速比
vl_values = [128, 256, 512, 1024]
results = {
    'VLEN': vl_values,
    '基线配置': [],
    'L1D 容量 x2': [],
    'L1D/L2 延迟 /2': []
}

for vlen in vl_values:
    # 基线配置
    results['基线配置'].append(calculate_speedup(df, vlen, 'RV', 'RVV'))
    # L1D 容量 x2
    results['L1D 容量 x2'].append(calculate_speedup(df, vlen, 'RV-L1D64KB', 'RVV-L1D64KB'))
    # L1D/L2 延迟 /2
    results['L1D/L2 延迟 /2'].append(calculate_speedup(df, vlen, 'RV-Latency/2', 'RVV-Latency/2'))

speedup_df = pd.DataFrame(results)
speedup_df = speedup_df.set_index('VLEN').T # 转置以匹配表格格式

print("RVV 加速比 (基线时间 / RVV 时间):")
print(speedup_df)


RVV 加速比 (基线时间 / RVV 时间):
VLEN              128       256       512       1024
基线配置          1.220851  1.050160  0.996041  0.997049
L1D 容量 x2     1.226014  1.053510  1.002174  0.996880
L1D/L2 延迟 /2  1.260530  1.086753  0.999388  0.997690


## 结果解释

下表显示了针对不同缓存配置和 VLEN 计算出的 RVV 加速比。加速比定义为相应配置下基线 RV 的执行时间除以 RVV 的执行时间。

您可以直接使用下表中的值来填充 LaTeX 表格：

- **基线配置** 行对应于 LaTeX 表格中的 `[基线, VLEN]` 值。
- **L1D 容量 x2** 行对应于 LaTeX 表格中的 `[L1Dx2, VLEN]` 值。
- **L1D/L2 延迟 /2** 行对应于 LaTeX 表格中的 `[延迟/2, VLEN]` 值。

In [23]:
# 以更适合填充表格的格式显示
print(speedup_df.round(3)) # 保留两位小数


VLEN           128    256    512    1024
基线配置          1.221  1.050  0.996  0.997
L1D 容量 x2     1.226  1.054  1.002  0.997
L1D/L2 延迟 /2  1.261  1.087  0.999  0.998
